# Activity Classification - DNN1 Training

This notebook trains our first Deep Neural Network on the physical activity dataset.


## Load Data and Prepare Training Set

In [2]:
!pip install kagglehub

  Using cached kagglehub-0.3.13-py3-none-any.whl.metadata (38 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached kagglehub-0.3.13-py3-none-any.whl (68 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [kagglehub]


In [6]:
%reset -f

import importlib
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import activity_functions
importlib.reload(activity_functions)
from activity_functions import *

In [7]:
activtity = load_data()

Loaded from Kaggle: /Users/tamimdostyar/.cache/kagglehub/datasets/diegosilvadefrana/fisical-activity-dataset/versions/4/dataset2.csv


In [8]:
df_train, df_test = create_train_test(activtity, test_ratio=0.2)
print(df_train.shape)
print(df_test.shape)

(2291244, 33)
(572812, 33)


In [9]:
X_train, y_train, X_test, y_test = prepare_for_train(df_train, df_test)

## HyperTune

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='dnn1'
)

tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

(2291244, 35)

## Best Hyperparameters

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
print(f"  n_hidden: {best_hps.get('n_hidden')}")
print(f"  n_neurons: {best_hps.get('n_neurons')}")
print(f"  activation: {best_hps.get('activation')}")
print(f"  learning_rate: {best_hps.get('learning_rate')}")
print(f"  optimizer: {best_hps.get('optimizer')}")

## Model Fit

In [ ]:

best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=20, validation_split=0.2)

## Model Evaluation

In [ ]:
y_test_hat = best_model.predict(X_test)
y_test_pred = np.argmax(y_test_hat, axis=1)
compute_scores(y_test, y_test_pred, verbose=True)


## Summary

Discuss results